#### Implementando RAG 
ChromaDB, Langchain e ChatGPT API

> Based on: https://medium.com/@drjulija/how-i-built-a-basic-rag-for-pdf-qa-in-a-few-lines-of-python-code-9849c32e59f0

In [2]:
# import libraries
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [34]:
# setar a variável de ambiente antes abrir o vscode
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') # add your OpenAI API Key
DOC_PATH = "./data/doc.pdf"
CHROMA_PATH = "./db" # ou user instalação do chromadb

In [35]:
# ----- Data Indexing Process -----

# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

# get OpenAI Embedding model
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# embed the chunks as vectors and load them into the database
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)


In [36]:
# ----- Retrieval and Generation Process -----

# this is an example of a user question (query)
query = 'what is this document about?'

# retrieve context - top 5 most relevant (closests) chunks to the query vector 
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

# you can use a prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a short answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

# call LLM model to generate the answer based on the given context and query
model = ChatOpenAI()

In [37]:
# LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7
# response_text = model.predict(prompt) 
# ver mais detalhes em https://python.langchain.com/v0.2/docs/versions/v0_2/deprecations/

response = model.invoke(prompt)
response_text = response.content

In [38]:
response_text

'Baby names, specifically the name Caio, its origin, meaning, and popularity.'